## NLP Continue

In [1]:
# getting the helper functions

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-06-11 15:15:27--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-06-11 15:15:27 (84.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

# Dataset

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2024-06-11 15:15:50--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.207, 173.194.79.207, 108.177.96.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  1.22MB/s    in 0.5s    

2024-06-11 15:15:50 (1.22 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
unzip_data("nlp_getting_started.zip")

# Visualize the dataset

In [6]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [7]:
train_df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
test_df.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df_shuffled = train_df.sample(frac=1, random_state=2)
train_df_shuffled.head()

,id,keyword,location,text,target
3190,4579,emergency%20plan,North Hastings Ontario,Practice your families fire escape plan so eve...,0
6171,8801,sirens,"Nomad, USA",Fuck Sleeping With Sirens.,0
1196,1722,buildings%20burning,NaN,'i'm a Gemini' *children screaming buildings b...,1
680,982,blazing,"Pig Symbol, Alabama",Montgomery come for the blazing hot weather......,1
3358,4808,evacuated,NaN,I got evacuated from the cinema 30 mins throug...,0


In [10]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [11]:
len(train_df), len(test_df)

(7613, 3263)

In [12]:
## Visualizing the random training examples

import random

random_index = random.randint(0, len(train_df) -5)

for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("_______________________________________________________________")

Target: 1 (real disaster)
Text:
Honestly tho Modibo Maiga is stealing a living - fuck all about him - im past my best but still more of a danger than that fucktard #coyi

_______________________________________________________________
Target: 0 (not real disaster)
Text:
LetÛªs talk some more about your goof guild Saunders. Come right up here on stage. https://t.co/hkBxxvd9Iw

_______________________________________________________________
Target: 0 (not real disaster)
Text:
Check out Vintage Longaberger Floral Fabric Shoulder Cross Body Bag Brown Leather Strap http://t.co/FB8snRg4HU @eBay

_______________________________________________________________
Target: 1 (real disaster)
Text:
25 killed as Kamayani Express Janata Express derail in Madhya Pradesh; ex gratia announced http://t.co/6SDTzSgElq

_______________________________________________________________
Target: 1 (real disaster)
Text:
only had a car for not even a week and got in a fucking car accident .. Mfs can't fucking drive

# Split data into training and validation dataset


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_sentences, val_sentences, train_labels , val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1,
    random_state=2
)

In [15]:
train_sentences.shape[0], val_sentences.shape[0]

(6851, 762)

In [16]:
train_labels.shape[0], val_labels.shape[0]

(6851, 762)

# Tokenization - tokenize the dataset

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [18]:
text_vectorizer = TextVectorization(
    max_tokens = None,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = None
)

In [19]:
split_1 = train_sentences[0].split()
len(split_1)

18

In [20]:
# Find the average number of tokens(words) in the training tweets

round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [21]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    output_sequence_length = max_length,
    pad_to_max_tokens = True
)

In [22]:
# fit the text vectorizer to the training text

text_vectorizer.adapt(train_sentences)

In [23]:
sample_sentence = "There is an apple tree in my garden."
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  75,    9,   41, 3156,  934,    4,   13, 2972,    0,    0,    0,
           0,    0,    0,    0]])>

In [24]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1406,   14, 2417, 1840, 2048, 1671, 1471, 1643, 2593, 1145, 1102,
        1262,    0,    0,    0]])>

In [25]:
words_in_vocab = text_vectorizer.get_vocabulary()

top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

In [27]:
print(f"Number of words in vocab: {len(words_in_vocab)}")

print(f"Top 10 vocab words: {top_5_words}")
print(f"Bottom 10 vocab words: {bottom_5_words}")

Number of words in vocab: 10000
Top 10 vocab words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 10 vocab words: ['palm', 'palinfoen', 'palestinian\x89Û', 'paleface', 'pale']


# Creating an Embedding using an Embedding Layer

In [28]:
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim = max_vocab_length,
    output_dim = 128,
    input_length = max_length
)

In [29]:
random_sentence = random.choice(train_sentences)

print(f"Original text:\n {random_sentence}\n\n")
print("Embedded Version:\n")

sample_embed = embedding(
    text_vectorizer([random_sentence])
)

sample_embed

Original text:
 @zhenghxn i tried 11 eyes akame ga kill and tokyo ghoul all damn bloody i dont dare watch????????


Embedded Version:



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 1.8709745e-02, -4.8400629e-02, -4.4042338e-02, ...,
         -4.5804620e-02,  2.8889801e-02,  3.7086044e-02],
        [ 9.6426159e-04,  3.5174835e-02, -2.9880155e-02, ...,
          5.6635588e-05, -4.1531336e-02,  4.2492185e-02],
        [-2.6856257e-02, -1.1003565e-02,  1.8112171e-02, ...,
          2.7878094e-02, -3.6402334e-02,  3.1230617e-02],
        ...,
        [-1.5717577e-02, -4.4629097e-02, -7.8744777e-03, ...,
         -2.3406029e-02, -3.9222203e-02,  2.9796217e-02],
        [-3.8550686e-02, -9.5130578e-03, -3.1491328e-02, ...,
          2.7710307e-02, -3.8646683e-03,  3.9198104e-02],
        [ 9.6426159e-04,  3.5174835e-02, -2.9880155e-02, ...,
          5.6635588e-05, -4.1531336e-02,  4.2492185e-02]]], dtype=float32)>

In [30]:
sample_embed.shape

# 1 sample
# 15 tokens
# 128 vector each

TensorShape([1, 15, 128])

# Visualizing learned Embeddings

In [31]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [34]:
# model_1 again for embedding

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)

model_1 = tf.keras.Model(inputs, outputs, name = "model_1_dense")

In [35]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [36]:
# model compile

model_1.compile(
    loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [37]:
model_1_history = model_1.fit(
    x = train_sentences,
    y = train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels)
)

Epoch 1/5
215/215 [==============================] - 9s 34ms/step - loss: 0.6106 - accuracy: 0.6976 - val_loss: 0.5406 - val_accuracy: 0.7441
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4415 - accuracy: 0.8205 - val_loss: 0.4685 - val_accuracy: 0.7953
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3469 - accuracy: 0.8624 - val_loss: 0.4667 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2836 - accuracy: 0.8927 - val_loss: 0.4673 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2364 - accuracy: 0.9110 - val_loss: 0.4952 - val_accuracy: 0.7861


In [46]:
# Get the weight matrix of embedding layer
# these are the numerical representations of each token in our training data

embed_weights = model_1.get_layer("embedding").get_weights()[0] # "emgedding_1 or embedding_2 depends on how many times it is trained look at summary for this."
embed_weights


array([[ 0.02978202, -0.02898342, -0.06357263, ...,  0.00255451,
        -0.03322324, -0.0300354 ],
       [ 0.01836982,  0.06126888,  0.01402926, ...,  0.02077009,
         0.01296788,  0.0246413 ],
       [ 0.04415895,  0.03152904, -0.07535241, ...,  0.04031039,
         0.05480219,  0.01790519],
       ...,
       [ 0.04500661, -0.01243194, -0.03228219, ..., -0.03539377,
         0.02483397,  0.01170387],
       [ 0.09584761,  0.0885841 , -0.01306073, ...,  0.0330134 ,
         0.0385146 ,  0.07814839],
       [ 0.03419611,  0.08258538, -0.04151829, ...,  0.07367618,
         0.00241481,  0.05239393]], dtype=float32)

In [47]:
len(embed_weights)

10000

In [49]:
embed_weights.shape

# (10000, 128)  for 10000 unique vocab it has unique embedding of dim 128
# for 10000 vocab each has its own embedding value of 128 dim [0.2,0.1,....,0.2]

(10000, 128)

# Tensorflow Projector tool to visualize embedding

https://www.tensorflow.org/text/guide/word_embeddings

In [50]:
# tensorflow projector tool
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()


In [51]:
# download from colab to upload to projector

try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## load vectors.tsx and metadata.tsv to the embedding projector

https://projector.tensorflow.org/

# Modelling

* Model 0: Naive Bayes ✅
* Model 1: Feed-Forward neural network (dense model) ✅

Completed model0 and model 1 in previous


* Model 2: LSTM
* Model 3: GRU
* Model 4: Bidirectional-LSTM
* Model 5: 1D Convolutional Neural Network
* Model 6: Tensorflow Hub Pretrained Feature Extractor (using transfer learnig for NLP)
* Model 7: Same as model 6 with 10% of training data

# Model 2: Using RNN LSTM

resource: Andrej Kaparthy unreasonable effectivenesss of recurrent neural networks

## Our structure of an RNN typically looks like this:

Input(text) -> Tokenize -> Embedding -> Layers (RNN/Dense) -> Output (label probability)

In [55]:
# creating an LSTM model

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
# shape = (batch, timesteps, feature)
x = layers.LSTM(64, return_sequences=True)(x) # 64 -> hidden units, outputs all the hidden states outputs
print(x.shape)
# here the required dims = 3 (None, 15, 64)
# required return_sequences to stack LSTM model here
x = layers.LSTM(64)(x)
# This will only output the last cell output: no return sequence so
print(x.shape)
x = layers.Dense(64, activation="relu")(x)
print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 15, 64)
(None, 64)
(None, 64)


In [56]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_6 (LSTM)               (None, 15, 64)            49408     
                                                                 
 lstm_7 (LSTM)               (None, 64)                33024     
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                      

In [57]:
# Compiling the model

model_2.compile(
    loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [58]:
# Fit the model

model_2_history = model_2.fit(
    x = train_sentences,
    y = train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels)
)

Epoch 1/5
215/215 [==============================] - 13s 43ms/step - loss: 0.2127 - accuracy: 0.9244 - val_loss: 0.7449 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1540 - accuracy: 0.9426 - val_loss: 0.7072 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1263 - accuracy: 0.9540 - val_loss: 0.6939 - val_accuracy: 0.7585
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1019 - accuracy: 0.9596 - val_loss: 1.0894 - val_accuracy: 0.7559
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0793 - accuracy: 0.9693 - val_loss: 1.2432 - val_accuracy: 0.7533


In [68]:
# 3valuate the model

model_2.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 1.2432 - accuracy: 0.7533


[1.2432160377502441, 0.7532808184623718]

In [69]:
model_2_pred_probs = model_2.predict(val_sentences)

24/24 [==============================] - 2s 6ms/step


In [70]:
model_2_pred_probs[:10]

array([[9.9996352e-01],
       [3.3083227e-02],
       [5.8896523e-02],
       [4.4395230e-05],
       [1.5590519e-04],
       [7.6456108e-06],
       [4.5257041e-01],
       [9.9998915e-01],
       [9.9999785e-01],
       [3.6928239e-01]], dtype=float32)

In [71]:
# converting model_2 pred probs to labels

model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [72]:
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0., 0., 1., 1., 0.], dtype=float32)>

In [73]:
## Creating a function to track evaluation methods
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate_preds(y_true, y_pred):
  """Evaluate your prediction

  parameters:
  y_true = true labels
  y_pred = predicted labels
  """
  accuracy = accuracy_score(y_true, y_pred) * 100
  precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average = "weighted")

  model_results = {
      "accuracy": accuracy,
      "precision": precision * 100,
      "recall": recall * 100,
      "f1": f1 * 100
  }
  return model_results

In [74]:
print(evaluate_preds(val_labels, model_2_preds))

{'accuracy': 75.32808398950131, 'precision': 75.44703546672051, 'recall': 75.32808398950131, 'f1': 75.15399877604601}


# Model 3: RNN GRU

same as LSTM but slightly different in the GRU cell

Another popular and effective RNN component is the GPU or Gated Recurrent Unit

Has similar features like an LSTM cell but has less parameters
optimized to work effectively as LSTM

In [83]:
# Build an RNN using GRU layer

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences=True)(x)
# if we want to stack GRU or LSTM each other we should use return sequences:True
x = layers.LSTM(64, return_sequences=True)(x)
# using GRU -> LSTM -> GRU
x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs3 = layers.Dense(1, activation="sigmoid")(x)

model_3_test = tf.keras.Model(inputs, outputs3, name="model_3_GRU")

In [84]:
model_3_test.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_9 (GRU)                 (None, 15, 64)            37248     
                                                                 
 lstm_12 (LSTM)              (None, 15, 64)            33024     
                                                                 
 gru_10 (GRU)                (None, 64)                24960     
                                                       

In [90]:
# Building model_3

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences=False)(x)
# This layer is need to fix the dimensions after output of GRU if return sequence True(None, 15, 54)
# x = layers.GlobalAveragePooling1D()(x)
outputs_3_final = layers.Dense(1, activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs, outputs_3_final, name="model_3_final_GRU")

In [91]:
model_3.summary()

Model: "model_3_final_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_14 (GRU)                (None, 64)                37248     
                                                                 
 dense_23 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
___________

In [92]:
# compiling the model
model_3.compile(
    loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [93]:
# fit model

model_3_history = model_3.fit(
    x = train_sentences,
    y = train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels)
)

Epoch 1/5
215/215 [==============================] - 14s 44ms/step - loss: 0.1547 - accuracy: 0.9416 - val_loss: 0.9647 - val_accuracy: 0.7454
Epoch 2/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0847 - accuracy: 0.9701 - val_loss: 0.9019 - val_accuracy: 0.7546
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0710 - accuracy: 0.9708 - val_loss: 1.0490 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0607 - accuracy: 0.9756 - val_loss: 1.2518 - val_accuracy: 0.7493
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0499 - accuracy: 0.9788 - val_loss: 1.5451 - val_accuracy: 0.7493


In [95]:
# making predictions

model_3_pred_probs = model_3.predict(val_sentences)

24/24 [==============================] - 2s 6ms/step


In [98]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))

In [99]:
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0., 1., 1., 1., 1.], dtype=float32)>

In [101]:
print(evaluate_preds(val_labels, model_3_pred))

{'accuracy': 74.93438320209974, 'precision': 74.92986837445862, 'recall': 74.93438320209974, 'f1': 74.93199649230681}


## using the model_3 test with GRU LSTM GRU stacking layer

In [102]:
model_3_test.compile(
    loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

model_3_test_history = model_3_test.fit(
    x = train_sentences,
    y = train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels)
)

Epoch 1/5
215/215 [==============================] - 18s 60ms/step - loss: 0.1119 - accuracy: 0.9604 - val_loss: 1.3152 - val_accuracy: 0.7441
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0514 - accuracy: 0.9774 - val_loss: 1.5216 - val_accuracy: 0.7454
Epoch 3/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0562 - accuracy: 0.9740 - val_loss: 1.6913 - val_accuracy: 0.7402
Epoch 4/5
215/215 [==============================] - 6s 28ms/step - loss: 0.0464 - accuracy: 0.9783 - val_loss: 1.7453 - val_accuracy: 0.7441
Epoch 5/5
215/215 [==============================] - 5s 25ms/step - loss: 0.0409 - accuracy: 0.9812 - val_loss: 1.7310 - val_accuracy: 0.7480


In [103]:
model_3_test_pred_probs = model_3_test.predict(val_sentences)

model_3_test_preds = tf.squeeze(tf.round(model_3_test_pred_probs))

model_3_test_preds[:10]

24/24 [==============================] - 2s 6ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0., 1., 1., 1., 1.], dtype=float32)>

In [104]:
print(evaluate_preds(val_labels, model_3_test_preds))

{'accuracy': 74.80314960629921, 'precision': 74.7715023397, 'recall': 74.80314960629921, 'f1': 74.75413394579567}


# Model 4: Bidirectional RNN



In [109]:
# Building BidirectionalRNN model

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
print(x.shape)
# This will give (None, 128) due to bidirectional layer
outputs_4 = layers.Dense(1, activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs, outputs_4, name="model_4_bidirectionalRNN")

(None, 128)


In [111]:
model_4.summary()

Model: "model_4_bidirectionalRNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_4 (Bidirecti  (None, 15, 128)           98816     
 onal)                                                           
                                                                 
 bidirectional_5 (Bidirecti  (None, 128)               74496     
 onal)                                                           
                                          

In [112]:
# compiling the model

model_4.compile(
    loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [113]:
# Fit model

model_4_history = model_4.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data=(val_sentences, val_labels)
)

Epoch 1/5
215/215 [==============================] - 21s 48ms/step - loss: 0.0919 - accuracy: 0.9730 - val_loss: 1.5215 - val_accuracy: 0.7428
Epoch 2/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0459 - accuracy: 0.9791 - val_loss: 1.4739 - val_accuracy: 0.7441
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0408 - accuracy: 0.9788 - val_loss: 1.5821 - val_accuracy: 0.7415
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0384 - accuracy: 0.9815 - val_loss: 1.6914 - val_accuracy: 0.7428
Epoch 5/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0409 - accuracy: 0.9799 - val_loss: 1.6888 - val_accuracy: 0.7428


In [114]:
model_4_pred_probs = model_4.predict(val_sentences)

24/24 [==============================] - 2s 8ms/step


In [115]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))

In [116]:
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0., 1., 1., 1., 0.], dtype=float32)>

In [117]:
print(evaluate_preds(val_labels, model_4_preds))

{'accuracy': 74.2782152230971, 'precision': 74.46474654211087, 'recall': 74.2782152230971, 'f1': 74.04348951491869}
